In [1]:
import jovian

In [2]:
jovian.commit(project='decisiontree', privacy='secret')

<IPython.core.display.Javascript object>

[jovian] Creating a new project "uppula-abhishek/decisiontree"
[jovian] Committed successfully! https://jovian.ai/uppula-abhishek/decisiontree


'https://jovian.ai/uppula-abhishek/decisiontree'

In [71]:
import opendatasets as od
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os



In [72]:
od.download('https://www.kaggle.com/jsphyg/weather-dataset-rattle-package')

Skipping, found downloaded files in ".\weather-dataset-rattle-package" (use force=True to force download)


In [73]:
os.listdir('weather-dataset-rattle-package')

['weatherAUS.csv']

In [74]:
raw_df = pd.read_csv('weather-dataset-rattle-package/weatherAUS.csv')

In [75]:
raw_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [104]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [77]:
raw_df.dropna(subset=['RainTomorrow'],inplace=True)

In [78]:
jovian.commit(project='decisiontree', privacy='secret')

<IPython.core.display.Javascript object>

[jovian] Updating notebook "uppula-abhishek/decisiontree" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/uppula-abhishek/decisiontree


'https://jovian.ai/uppula-abhishek/decisiontree'

In [79]:
year = pd.to_datetime(raw_df.Date).dt.year

train_df = raw_df[year<2015]
val_df = raw_df[year == 2015]
test_df = raw_df[year > 2015]

In [80]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(98988, 23)
(17231, 23)
(25974, 23)


In [81]:
input_cols = list(train_df.columns)[1:-1]
target_col = 'RainTomorrow'

In [82]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

In [83]:
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

In [84]:
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [85]:
numeric_cols = train_inputs.select_dtypes(include = np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [86]:
numeric_cols

['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm']

In [87]:
categorical_cols

['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [88]:
train_df[numeric_cols].isna().sum().sort_values(ascending=False)

Sunshine         40696
Evaporation      37110
Cloud3pm         36766
Cloud9am         35764
Pressure9am       9345
Pressure3pm       9309
WindGustSpeed     6902
Humidity9am       1265
Humidity3pm       1186
WindSpeed3pm      1140
WindSpeed9am      1133
Rainfall          1000
Temp9am            783
Temp3pm            663
MinTemp            434
MaxTemp            198
dtype: int64

In [89]:
from sklearn.impute import SimpleImputer

In [90]:
imputer = SimpleImputer(strategy='mean').fit(raw_df[numeric_cols])

In [91]:
train_inputs[numeric_cols] = imputer.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = imputer.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = imputer.transform(test_inputs[numeric_cols])

In [92]:
from sklearn.preprocessing import MinMaxScaler

In [93]:
scaler = MinMaxScaler().fit(raw_df[numeric_cols])

In [94]:
train_inputs[numeric_cols].describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000,98988.000000
mean,12.002823,23.003328,2.372704,5.355452,7.611475,40.213769,14.101045,18.768900,68.654275,51.501499,1017.530858,1015.149778,4.354706,4.447402,16.828607,21.526705
std,6.331572,6.983092,8.475680,3.125083,2.907069,13.226474,8.942506,8.826241,18.873780,20.618122,6.731053,6.660441,2.292363,2.136544,6.374509,6.809602
min,-8.500000,-4.100000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,979.000000,0.000000,0.000000,-5.900000,-5.100000
25%,7.500000,17.900000,0.000000,3.800000,7.500000,31.000000,7.000000,13.000000,57.000000,37.000000,1013.400000,1010.900000,3.000000,3.000000,12.200000,16.600000
50%,11.800000,22.400000,0.000000,5.469824,7.624853,39.000000,13.000000,19.000000,69.000000,52.000000,1017.653758,1015.258204,4.437189,4.503167,16.600000,21.000000
75%,16.600000,27.900000,0.800000,5.600000,9.200000,46.000000,19.000000,24.000000,83.000000,65.000000,1021.800000,1019.300000,6.000000,6.000000,21.300000,26.100000
max,33.900000,48.100000,371.000000,82.400000,14.300000,135.000000,87.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.100000


In [95]:
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [96]:
from sklearn.preprocessing import OneHotEncoder

In [97]:
train_df[categorical_cols] = train_df[categorical_cols].fillna('Unknown')
val_df[categorical_cols]  = val_df[categorical_cols].fillna('Unknown')
test_df[categorical_cols] = test_df[categorical_cols].fillna('Unknown')


C:\Users\swayam\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [98]:
train_df[categorical_cols].isna().sum()

Location       0
WindGustDir    0
WindDir9am     0
WindDir3pm     0
RainToday      0
dtype: int64

In [99]:
encoder = OneHotEncoder( sparse=True,handle_unknown='ignore').fit(train_inputs[categorical_cols])

In [100]:
encoder_cols = list(encoder.get_feature_names(categorical_cols))

In [108]:
len(encoder_cols)

103

In [109]:
train_inputs[encoder_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoder_cols] = encoder.transform(val_inputs[categorical_cols])
test_inputs[encoder_cols] = encoder.transform(test_inputs[categorical_cols])

IndexError: tuple index out of range

In [110]:
encoder.transform(train_inputs[categorical_cols])

<98988x103 sparse matrix of type '<class 'numpy.float64'>'
	with 494940 stored elements in Compressed Sparse Row format>